In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy.stats import pearsonr

In [4]:
gene_1=pd.read_csv('https://schulzlab.github.io/teaching/algoepi2025/TrainingData/ENSG00000074803_train.txt',delimiter='\t', low_memory=False)
gene_2=pd.read_csv('https://schulzlab.github.io/teaching/algoepi2025/TrainingData/ENSG00000274090_train.txt',delimiter='\t', low_memory=False)
gene_3=pd.read_csv('https://schulzlab.github.io/teaching/algoepi2025/TrainingData/ENSG00000200121_train.txt',delimiter='\t', low_memory=False)


Preprocessing of all genes

In [5]:
def preprocess_gene_data(df):
    # This function performs log2 transformation, train-test split, and z-score scaling.

    df = df.copy()
    # Log-transformation of expression
    df['Expression'] = np.log2(df['Expression'] + 1)

    # Split into features (X) and target (y)
    # We ignore the first column (Sample-ID) and the last (Expression).
    X = df.iloc[:, 1:-1].to_numpy()
    y = df.iloc[:, -1].to_numpy()

    # Train-Test-Split with 'random_state=42' for reproducibility
    X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Scaling (Z-score)
    x_scaler = sk.preprocessing.StandardScaler()
    y_scaler = sk.preprocessing.StandardScaler()

    X_train_scaled = x_scaler.fit_transform(X_train)
    X_test_scaled = x_scaler.transform(X_test)
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))[:, 0]
    y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))[:, 0]

    # The function returns a dictionary to pass all processed data and the y_scaler structurally.
    return {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train_scaled,
        'y_test': y_test_scaled,
        'y_scaler': y_scaler
    }

# Prepare all three genes by calling the 'preprocess_gene_data' function for each dataset.
data_g1 = preprocess_gene_data(gene_1)
data_g2 = preprocess_gene_data(gene_2)
data_g3 = preprocess_gene_data(gene_3)

Linear model as base line

In [10]:
from sklearn.linear_model import LinearRegression

def run_linear_regression(data_dict, name):
    model = LinearRegression()
    model.fit(data_dict['X_train'], data_dict['y_train'])

    y_pred = model.predict(data_dict['X_test'])

    rmse = np.sqrt(sk.metrics.mean_squared_error(data_dict['y_test'], y_pred))
    corr, _ = pearsonr(data_dict['y_test'], y_pred)

    print(f"{name}")
    print(f"RMSE: {rmse:.4f} | Pearson: {corr:.4f}\n")
    return model

# run linear models
model_g1 = run_linear_regression(data_g1, "Gene 1")
model_g2 = run_linear_regression(data_g2, "Gene 2")
model_g3 = run_linear_regression(data_g3, "Gene 3")

Gene 1
RMSE: 0.9203 | Pearson: 0.5114

Gene 2
RMSE: 1.2932 | Pearson: 0.5531

Gene 3
RMSE: 0.8039 | Pearson: 0.7110



svm model test

In [13]:
from sklearn.svm import SVR
import pandas as pd

datasets = [
    (data_g1, "Gene 1"),
    (data_g2, "Gene 2"),
    (data_g3, "Gene 3")
]

svr_results = []

# loop across all genes
for data, name in datasets:
    # C=1.0 and epsilon=0.1 as standard values
    model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

    # training the model
    model.fit(data['X_train'], data['y_train'])

    # predict test data
    y_pred = model.predict(data['X_test'])

    # evaluation
    rmse = np.sqrt(sk.metrics.mean_squared_error(data['y_test'], y_pred))
    corr, _ = pearsonr(data['y_test'], y_pred)

    print(f"{name}")
    print(f"RMSE: {rmse:.4f} | Pearson: {corr:.4f}\n")


Gene 1
RMSE: 0.5998 | Pearson: 0.7340

Gene 2
RMSE: 0.8539 | Pearson: 0.2018

Gene 3
RMSE: 0.5225 | Pearson: 0.8680



Conclusion: SVR for gene 1 and 3 better, but worse for gene 2: why?

grid search for better model parameters for svr

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd

# selection of values for parameters to test
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2],
    'gamma': ['scale', 0.001, 0.01]
}

results = []
datasets = [(data_g1, "Gene 1"), (data_g2, "Gene 2"), (data_g3, "Gene 3")]

# gridsearch for best parameters with 5 fold cross validation
for data, name in datasets:
    grid = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid.fit(data['X_train'], data['y_train'])

    # use best prediction for model
    best_svr = grid.best_estimator_
    y_pred = best_svr.predict(data['X_test'])

    # checking of estimation
    rmse = np.sqrt(sk.metrics.mean_squared_error(data['y_test'], y_pred))
    corr, _ = pearsonr(data['y_test'], y_pred)

    print(f"{name}")
    print(f"best parameters: {grid.best_params_}")
    print(f"RMSE: {rmse:.4f} | Pearson: {corr:.4f}\n")

Gene 1
best parameters: {'C': 10, 'epsilon': 0.01, 'gamma': 'scale'}
RMSE: 0.5528 | Pearson: 0.7540

Gene 2
best parameters: {'C': 100, 'epsilon': 0.1, 'gamma': 0.001}
RMSE: 0.7565 | Pearson: 0.5016

Gene 3
best parameters: {'C': 1, 'epsilon': 0.01, 'gamma': 'scale'}
RMSE: 0.5219 | Pearson: 0.8695



-- feature selection to optimize svr model